<a href="https://colab.research.google.com/github/NguyenCuuNguyen/ChatGPT_sandbox/blob/main/Bert_Not_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
dir = '/content/drive/MyDrive/ML_output_scrambled.csv'

#https://medium.com/geekculture/simple-chatbot-using-bert-and-pytorch-part-1-2735643e0baa

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Install Transformers
!pip install transformers
# To get model summary
!pip install torchinfo


In [ ]:
import numpy as np
import pandas as pd
import re
import torch
import random
import torch.nn as nn
import transformers
import matplotlib.pyplot as plt
# specify GPU
device = torch.device('cuda')

In [ ]:
import pandas as pd
import csv
df = pd.read_csv(dir)
df.head()

,CONTENT,lang,latitude,longitude,created_at,COUNTRY,id_str,cleaned_content,target,color
0,私達が子供のときは新型コロナウイルスなんて存在しなかったし、\nワクチンがなかったので、ヒブ...,ja,35.689456,139.691716,Thu Jul 21 00:19:55 +0000 2022,Japan,1549912016466640896,子供 新型 コロナ ウイルス 存在 ワクチン 肺炎 球菌 髄膜 子供 命 落とす 麻疹 ワク...,Anti-Vax,red
1,私達が子供のときは新型コロナウイルスなんて存在しなかったし、\nワクチンがなかったので、ヒブ...,ja,35.089432,137.074866,Thu Jul 21 00:20:34 +0000 2022,Japan,1549912178635214849,子供 新型 コロナ ウイルス 存在 ワクチン 肺炎 球菌 髄膜 子供 命 落とす 麻疹 ワク...,Anti-Vax,red
2,私達が子供のときは新型コロナウイルスなんて存在しなかったし、\nワクチンがなかったので、ヒブ...,ja,35.180226,136.906739,Thu Jul 21 00:18:24 +0000 2022,Japan,1549911636978573313,子供 新型 コロナ ウイルス 存在 ワクチン 肺炎 球菌 髄膜 子供 命 落とす 麻疹 ワク...,Anti-Vax,red
3,私達が子供のときは新型コロナウイルスなんて存在しなかったし、\nワクチンがなかったので、ヒブ...,ja,39.852086,141.164859,Thu Jul 21 00:18:12 +0000 2022,Japan,1549911583991955456,子供 新型 コロナ ウイルス 存在 ワクチン 肺炎 球菌 髄膜 子供 命 落とす 麻疹 ワク...,Anti-Vax,red
4,私達が子供のときは新型コロナウイルスなんて存在しなかったし、\nワクチンがなかったので、ヒブ...,ja,35.607474,140.106628,Thu Jul 21 00:17:47 +0000 2022,Japan,1549911478597459968,子供 新型 コロナ ウイルス 存在 ワクチン 肺炎 球菌 髄膜 子供 命 落とす 麻疹 ワク...,Anti-Vax,red


In [ ]:
df['target'].value_counts()
# Converting the labels into encodings
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['target'] = le.fit_transform(df['target'])
# check class distribution
df['target'].value_counts(normalize = True)

0    0.582669
2    0.382539
1    0.034791
Name: target, dtype: float64

In [ ]:
train_text, train_labels = df['CONTENT'], df['target']

In [ ]:
# from transformers import AutoModel, BertTokenizerFast
# # Load the BERT tokenizer
# tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
# # Import BERT-base pretrained model
# bert = AutoModel.from_pretrained('bert-base-uncased')

In [ ]:
# from transformers import RobertaTokenizer, RobertaModel
# # Load the Roberta tokenizer
# tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
# # Import Roberta pretrained model
# bert = RobertaModel.from_pretrained('roberta-base')

In [ ]:
from transformers import DistilBertTokenizer, DistilBertModel
# Load the DistilBert tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('karolill/distilmbert_LR3e-05_WR0.1_OPTIMadamw_hf_WD0.01')#('distilbert-base-uncased')
# Import the DistilBert pretrained model
bert = DistilBertModel.from_pretrained('karolill/distilmbert_LR3e-05_WR0.1_OPTIMadamw_hf_WD0.01')#('distilbert-base-uncased')

In [ ]:
#max_seq_len is the longest lenght of 1 data point
tokens_train = tokenizer(
    train_text.tolist(),
    max_length = 512,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2393: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
#Convert int sequence to tensor
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_labels.tolist())

create dataloaders for the training set. These dataloaders will pass batches of train data as input to the model during the training phase.

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
#define a batch size
batch_size = 16
# wrap tensors
train_data = TensorDataset(train_seq, train_mask, train_y)
# sampler for sampling the data during training
train_sampler = RandomSampler(train_data)
# DataLoader for train set
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

In [ ]:
class BERT_Arch(nn.Module):
   def __init__(self, bert):
       super(BERT_Arch, self).__init__()
       self.bert = bert

       # dropout layer
       self.dropout = nn.Dropout(0.2)

       # relu activation function
       self.relu =  nn.ReLU()
       # dense layer
       self.fc1 = nn.Linear(768,512)
       self.fc2 = nn.Linear(512,256)
       self.fc3 = nn.Linear(256,3)
       #softmax activation function
       self.softmax = nn.LogSoftmax(dim=1)
       #define the forward pass
   def forward(self, sent_id, mask):
      #pass the inputs to the model
      cls_hs = self.bert(sent_id, attention_mask=mask)[0][:,0]

      x = self.fc1(cls_hs)
      x = self.relu(x)
      x = self.dropout(x)

      x = self.fc2(x)
      x = self.relu(x)
      x = self.dropout(x)
      # output layer
      x = self.fc3(x)

      # apply softmax activation
      x = self.softmax(x)
      return x

In [ ]:
# freeze all the parameters. This will prevent updating of model weights during fine-tuning.
for param in bert.parameters():
      param.requires_grad = False
model = BERT_Arch(bert)
# push the model to GPU
model = model.to(device)
from torchinfo import summary
summary(model)

Layer (type:depth-idx)                                  Param #
BERT_Arch                                               --
├─DistilBertModel: 1-1                                  --
│    └─Embeddings: 2-1                                  --
│    │    └─Embedding: 3-1                              (91,812,096)
│    │    └─Embedding: 3-2                              (393,216)
│    │    └─LayerNorm: 3-3                              (1,536)
│    │    └─Dropout: 3-4                                --
│    └─Transformer: 2-2                                 --
│    │    └─ModuleList: 3-5                             (42,527,232)
├─Dropout: 1-2                                          --
├─ReLU: 1-3                                             --
├─Linear: 1-4                                           393,728
├─Linear: 1-5                                           131,328
├─Linear: 1-6                                           771
├─LogSoftmax: 1-7                                       --
Total pa

In [ ]:
#Using the Optimizer we reduce the loss during backpropagation through the network.
from transformers import AdamW
# define the optimizer
optimizer = AdamW(model.parameters(), lr = 1e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Balancing the weights while calculating the error

In [ ]:
import sklearn
sklearn.__version__

'1.2.2'

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
#compute the class weights
class_wts = compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(train_labels),
                                        y = train_labels
                                    )
print(class_wts)

[0.57207961 9.58099688 0.87136988]


In [ ]:
#Balancing the weights while calculating the error
# convert class weights to tensor
weights= torch.tensor(class_wts,dtype=torch.float)
weights = weights.to(device)
# loss function
cross_entropy = nn.NLLLoss(weight=weights)

In [ ]:
#Setting up the epochs
# empty lists to store training and validation loss of each epoch
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
train_losses=[]
# number of training epochs
epochs = 200
# We can also use learning rate scheduler to achieve better results
lr_sch = lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1)

Finetune model

In [ ]:
# function to train the model
def train():

  model.train()
  total_loss = 0

  # empty list to save model predictions
  total_preds=[]

  # iterate over batches
  for step,batch in enumerate(train_dataloader):

    # progress update after every 50 batches.
    if step % 50 == 0 and not step == 0:
      print('  Batch {:>5,}  of  {:>5,}.'.format(step,    len(train_dataloader)))
    # push the batch to gpu
    batch = [r.to(device) for r in batch]
    sent_id, mask, labels = batch
    # get model predictions for the current batch
    preds = model(sent_id, mask)
    # compute the loss between actual and predicted values
    loss = cross_entropy(preds, labels)
    # add on to the total loss
    total_loss = total_loss + loss.item()
    # backward pass to calculate the gradients
    loss.backward()
    # clip the the gradients to 1.0. It helps in preventing the    exploding gradient problem
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    # update parameters
    optimizer.step()
    # clear calculated gradients
    optimizer.zero_grad()

    # We are not using learning rate scheduler as of now
    # lr_sch.step()
    # model predictions are stored on GPU. So, push it to CPU
    preds=preds.detach().cpu().numpy()
    # append the model predictions
    total_preds.append(preds)
    # compute the training loss of the epoch
    avg_loss = total_loss / len(train_dataloader)

    # predictions are in the form of (no. of batches, size of batch, no. of classes).
    # reshape the predictions in form of (number of samples, no. of classes)
    total_preds  = np.concatenate(total_preds, axis=0)
    #returns the loss and predictions
    return avg_loss, total_preds

In [ ]:
for epoch in range(epochs):

    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))

    #train model
    train_loss, _ = train()

    # append training and validation loss
    train_losses.append(train_loss)
    # it can make your experiment reproducible, similar to set  random seed to all options where there needs a random seed.
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print(f'\nTraining Loss: {train_loss:.3f}')


 Epoch 1 / 200

Training Loss: 0.004

 Epoch 2 / 200

Training Loss: 0.003

 Epoch 3 / 200

Training Loss: 0.001

 Epoch 4 / 200

Training Loss: 0.004

 Epoch 5 / 200

Training Loss: 0.001

 Epoch 6 / 200

Training Loss: 0.001

 Epoch 7 / 200

Training Loss: 0.004

 Epoch 8 / 200

Training Loss: 0.001

 Epoch 9 / 200

Training Loss: 0.002

 Epoch 10 / 200

Training Loss: 0.004

 Epoch 11 / 200

Training Loss: 0.006

 Epoch 12 / 200

Training Loss: 0.002

 Epoch 13 / 200

Training Loss: 0.003

 Epoch 14 / 200

Training Loss: 0.001

 Epoch 15 / 200

Training Loss: 0.004

 Epoch 16 / 200

Training Loss: 0.002

 Epoch 17 / 200

Training Loss: 0.001

 Epoch 18 / 200

Training Loss: 0.004

 Epoch 19 / 200

Training Loss: 0.002

 Epoch 20 / 200

Training Loss: 0.007

 Epoch 21 / 200

Training Loss: 0.003

 Epoch 22 / 200

Training Loss: 0.005

 Epoch 23 / 200

Training Loss: 0.001

 Epoch 24 / 200

Training Loss: 0.001

 Epoch 25 / 200

Training Loss: 0.004

 Epoch 26 / 200

Training Loss: 0

In [ ]:
model = model.cpu()
torch.save(model, "bert_model.pt")
model = model.cuda()

In [ ]:
def get_prediction(str):
  str = re.sub(r'[^a-zA-Z ]+', '', str)
  test_text = [str]
  model.eval()

  tokens_test_data = tokenizer(
  test_text,
  max_length = 512,
  pad_to_max_length=True,
  truncation=True,
  return_token_type_ids=False
  )
  test_seq = torch.tensor(tokens_test_data['input_ids'])
  test_mask = torch.tensor(tokens_test_data['attention_mask'])

  preds = None
  with torch.no_grad():
    preds = model(test_seq.to(device), test_mask.to(device))
  preds = preds.detach().cpu().numpy()
  preds = np.argmax(preds, axis = 1)
  print('Intent Identified: ', le.inverse_transform(preds)[0])
  return le.inverse_transform(preds)[0]


def get_response(message):
  intent = get_prediction(message)
  for i in df['target']:
    if i["tag"] == intent:
      result = random.choice(i["target"])
      break
  print(f"Response : {result}")
  return "Intent: "+ intent + '\n' + "Response: " + result

In [ ]:
new_model = torch.load("bert_model.pt")

In [ ]:
#NEED TO ADD TAG TO THE COLUMNS
df["target"] = [[0,1,2]]*len(df)
get_prediction("ワクチンが好きです")
get_prediction("I like vaccine")

Intent Identified:  Pro-Vax
Intent Identified:  Anti-Vax


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2393: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2393: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input 

'Anti-Vax'

In [ ]:
df.head()

Add test data to get new predictions

In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/Tweepy_data_clean.csv')
test_df.head()


,content,created_at,user_location,id_str
0,.@MLongMD with great comprehensive #immunizati...,Tue Feb 22 21:43:52 +0000 2022,"Jacksonville, FL",1.496240e+18
1,PSA: Let’s get rid of the stigma surrounding #...,Tue Feb 22 18:58:10 +0000 2022,"Perth, Western Australia",1.496200e+18
2,@MartianPontiac @NoyesJHumphrey @JanPeterBalke...,Tue Feb 22 19:12:35 +0000 2022,New York,1.496200e+18
3,SJM has strong reasons to believe that HPV may...,Tue Feb 22 17:54:21 +0000 2022,"Delhi, India",1.496180e+18
4,Henrietta Lacks cells(HeLa cells) were taken u...,Tue Feb 22 18:56:17 +0000 2022,Alaska,1.496200e+18
